# DuckGuard - Data Quality in 60 Seconds

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/XDataHubAI/duckguard/blob/main/examples/colab_quickstart.ipynb)
[![PyPI](https://img.shields.io/pypi/v/duckguard.svg)](https://pypi.org/project/duckguard/)

**DuckGuard** is a Python-native data quality tool built on DuckDB. 10x faster than pandas-based tools.

Features:
- Quality Scoring (A-F grades)
- YAML-based Rules
- Semantic Type Detection (PII, emails, etc.)
- Data Contracts
- Anomaly Detection

In [ ]:
# Install DuckGuard
!pip install duckguard -q
print("DuckGuard installed!")

In [ ]:
# Create sample data
import pandas as pd

df = pd.DataFrame({
    'order_id': ['ORD-001', 'ORD-002', 'ORD-003', 'ORD-004', 'ORD-005'],
    'customer_id': ['CUST-001', 'CUST-002', None, 'CUST-004', 'CUST-005'],
    'email': ['john@example.com', 'jane@example.com', 'bob@example.com', 'invalid-email', 'alice@example.com'],
    'amount': [99.99, 149.50, 75.00, -10.00, 200.00],
    'status': ['delivered', 'shipped', 'pending', 'unknown', 'delivered']
})

df.to_csv('orders.csv', index=False)
print("Sample data created!")
df

## 1. Connect and Explore

In [ ]:
from duckguard import connect

# Connect to data
orders = connect("orders.csv")

print(f"Rows: {orders.row_count}")
print(f"Columns: {orders.columns}")

## 2. Quality Score

In [ ]:
# Get instant quality score
result = orders.score()

print(f"Quality Score: {result.overall:.1f}/100")
print(f"Grade: {result.grade}")
print("\nDimensions:")
print(f"  Completeness: {result.completeness:.1f}")
print(f"  Uniqueness: {result.uniqueness:.1f}")
print(f"  Validity: {result.validity:.1f}")

## 3. Column Statistics

In [ ]:
# Check column quality
print(f"customer_id null %: {orders.customer_id.null_percent:.1f}%")
print(f"order_id unique %: {orders.order_id.unique_percent:.1f}%")
print(f"amount min: {orders.amount.min}")
print(f"amount max: {orders.amount.max}")

## 4. Semantic Type Detection (PII)

In [ ]:
from duckguard import detect_types_for_dataset
from duckguard.semantic import SemanticAnalyzer

# Detect semantic types
types = detect_types_for_dataset(orders)
for col, sem_type in types.items():
    print(f"{col}: {sem_type.value if sem_type else 'generic'}")

# Check for PII
analysis = SemanticAnalyzer().analyze(orders)
if analysis.pii_columns:
    print(f"\n⚠️ PII detected in: {analysis.pii_columns}")

## 5. YAML Rules

In [ ]:
from duckguard import execute_rules, load_rules_from_string

yaml_rules = """
dataset: orders
rules:
  - order_id is not null
  - order_id is unique
  - customer_id null_percent < 50
  - amount >= 0
  - status in ['pending', 'shipped', 'delivered']
"""

rules = load_rules_from_string(yaml_rules)
result = execute_rules(rules, dataset=orders)

print(f"Passed: {result.passed_count}/{result.total_checks}")
print("\nResults:")
for r in result.results:
    status = "✓" if r.passed else "✗"
    print(f"  {status} {r.check.expression}")

## 6. Anomaly Detection

In [ ]:
from duckguard import detect_anomalies

report = detect_anomalies(orders, method="zscore", threshold=2.0)

print(f"Anomalies found: {report.anomaly_count}")
for a in report.anomalies:
    if a.is_anomaly:
        print(f"  ⚠️ {a.column}: {a.message}")

## 7. Data Contracts

In [ ]:
from duckguard import generate_contract, validate_contract

# Generate contract from data
contract = generate_contract(orders, name="orders_contract")

print(f"Contract: {contract.name}")
print("Schema:")
for field in contract.schema:
    print(f"  {field.name}: {field.type.value}")

# Validate
result = validate_contract(contract, orders)
print(f"\nValid: {result.is_valid}")

## Next Steps

- **GitHub**: https://github.com/XDataHubAI/duckguard
- **PyPI**: https://pypi.org/project/duckguard/
- **Full docs**: See `examples/getting_started.ipynb`

```bash
# CLI usage
duckguard check data.csv
duckguard discover data.csv --output rules.yaml
duckguard anomaly data.csv
```

In [ ]:
# Real-world validation workflow
print("=" * 60)
print("DUCKGUARD 3.0 - COMPREHENSIVE VALIDATION")
print("=" * 60)

checks = {}

# 1. Conditional: High-value orders need customer info
checks['conditional'] = orders.customer_id.not_null_when("amount >= 100")
print(f"\n✓ Conditional check: {checks['conditional'].passed}")

# 2. Multi-column: Validate relationships
checks['multicolumn'] = orders.expect_column_pair_satisfy(
    column_a="amount",
    column_b="status",
    expression="amount >= 0 OR status = 'unknown'",
    threshold=1.0
)
print(f"✓ Multi-column check: {checks['multicolumn'].passed}")

# 3. Query-based: Business logic validation
checks['query'] = orders.expect_query_to_return_no_rows(
    query="SELECT * FROM table WHERE status = 'delivered' AND customer_id IS NULL"
)
print(f"✓ Query-based check: {checks['query'].passed}")

# Summary
passed = sum(1 for c in checks.values() if c.passed)
print(f"\n{'='*60}")
print(f"RESULTS: {passed}/{len(checks)} checks passed")
print(f"{'='*60}")

## 🎯 Putting It All Together: 3.0 Power Move

Combine all 3.0 features for sophisticated validation:

**Distributional test methods:**
- `expect_distribution_normal()` - Test for normal distribution
- `expect_distribution_uniform()` - Test for uniform distribution  
- `expect_ks_test(distribution)` - Kolmogorov-Smirnov test for any distribution
- `expect_chi_square_test(expected_frequencies)` - Chi-square goodness-of-fit test

These are perfect for validating ML features or detecting data drift!

In [ ]:
# Install scipy for distributional tests
!pip install scipy -q

# Test if amount follows a specific distribution
try:
    result = orders.amount.expect_ks_test(
        distribution='norm',  # Normal distribution
        significance_level=0.05
    )

    print(f"Normality test: {result.passed}")
    print(f"P-value: {result.details.get('pvalue', 0):.4f}")
    print("\nInterpretation:")
    if result.passed:
        print("  Data appears to follow a normal distribution")
    else:
        print("  Data does not follow a normal distribution")

except ImportError:
    print("scipy not available - skipping distributional tests")

## 🆕 New in 3.0: Distributional Testing

Test if your data follows specific statistical distributions (requires scipy):

**Query-based check methods:**
- `expect_query_to_return_no_rows(query)` - Query should return empty result
- `expect_query_to_return_rows(query)` - Query should return at least one row
- `expect_query_result_to_equal(query, expected)` - Query result equals specific value
- `expect_query_result_to_be_between(query, min, max)` - Query result in range

All queries use the keyword `table` to reference your dataset.

In [ ]:
# Check 1: Verify no invalid orders (query should return 0 rows)
result = orders.expect_query_to_return_no_rows(
    query="SELECT * FROM table WHERE status = 'unknown' AND amount > 0"
)

print(f"No invalid orders: {result.passed}")
print(f"Message: {result.message}")

# Check 2: Verify we have data (query should return rows)
result2 = orders.expect_query_to_return_rows(
    query="SELECT * FROM table WHERE status = 'delivered'"
)

print(f"\nHave delivered orders: {result2.passed}")

# Check 3: Verify specific metric equals expected value
result3 = orders.expect_query_result_to_equal(
    query="SELECT COUNT(*) FROM table WHERE amount < 0",
    expected=1  # We have exactly 1 negative amount in sample data
)

print(f"\nExact count check: {result3.passed}")

## 🆕 New in 3.0: Query-Based Checks

Write custom SQL queries to validate complex business logic:

In [ ]:
# Example: Verify amount is always positive
result = orders.expect_column_pair_satisfy(
    column_a="amount",
    column_b="amount",  # Can reference same column
    expression="amount >= 0",
    threshold=1.0  # 100% must pass
)

print(f"Check: {result.message}")
print(f"Passed: {result.passed}")

# More complex: Check that negative amounts only occur in specific status
result2 = orders.expect_column_pair_satisfy(
    column_a="amount",
    column_b="status",
    expression="amount >= 0 OR status = 'unknown'",
    threshold=1.0
)

print(f"\nComplex check: {result2.passed}")

## 🆕 New in 3.0: Multi-Column Validation

Validate relationships between multiple columns using SQL expressions:

**Other conditional checks available:**
- `not_null_when(condition)` - Column must not be null when condition is true
- `unique_when(condition)` - Column must be unique when condition is true
- `between_when(min, max, condition)` - Column must be in range when condition is true
- `isin_when(values, condition)` - Column must be in list when condition is true
- `pattern_when(regex, condition)` - Column must match pattern when condition is true

In [ ]:
# Conditional validation: customer_id required when amount > 100
result = orders.customer_id.not_null_when("amount >= 100")

print(f"Check passed: {result.passed}")
print(f"Message: {result.message}")
print("\nDetails:")
print(f"  Matching rows: {result.details.get('matching_rows', 0)}")
print(f"  Violations: {result.details.get('violations', 0)}")

## 🆕 New in 3.0: Conditional Validation

Sometimes you need to validate a column only when certain conditions are met. DuckGuard 3.0 introduces **conditional expectations** with the `when` clause: